In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import math as m
import pickle

# Needed to classify the data (not regression)
from sklearn.tree import DecisionTreeClassifier

# Used to assign values in a column to a # 
from sklearn.preprocessing import LabelEncoder 

# Splits data into training and testing datasets 
from sklearn.model_selection import train_test_split

# The metrics is used to calculate the accuracy of the model
from sklearn import metrics

In [2]:
import warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [6]:
# Data for 2005 to 2010 
# dataset = pd.read_csv('../Data/features_2000-2010.csv')
dataset = pd.read_csv('../Data/features_2000_2010_referenced.csv')
# Data for 2010
id_2010 = pd.read_csv('../Data/papers2010.csv')
dataset.head()

,Unnamed: 0,id,diversity,venue_rank,venue_MPI,venue_TPI,productivity,H_index,author_rank,author_MPI,...,versatility,n_citation,referenced_by,referenced_year,citation,citation_diff,citation_1yr,citation_2yr,citation_5yr,citation_10yr
0,0,1388,1.390561,11859.0,302,7688,3.0,2.0,333158.00,38.0,...,2.757852,1,[1976644430],[2011],1.0,0.0,0.0,0.0,0.0,0.0
1,1,49268,1.677395,11859.0,302,7688,170.0,30.0,351701.50,194.0,...,10.353863,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,209002,1.068871,11859.0,302,7688,1.0,0.0,1385029.00,0.0,...,1.068871,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,252754,1.083049,11859.0,302,7688,114.5,25.0,312797.75,194.0,...,9.060297,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,526827,0.722842,11859.0,302,7688,1.0,1.0,642772.50,8.0,...,0.722842,8,"[1558160053, 1981550344, 1998786313, 2952127479]","[2002, 2008, 2009, 2012]",4.0,4.0,1.0,1.0,1.0,4.0


### Applying train and test split 

In [7]:
# Split data for train and test 
ids = id_2010['id'].tolist() 
train = dataset[dataset['id'].isin(ids) == False]
test = dataset[dataset['id'].isin(ids)]
train.head()

,Unnamed: 0,id,diversity,venue_rank,venue_MPI,venue_TPI,productivity,H_index,author_rank,author_MPI,...,versatility,n_citation,referenced_by,referenced_year,citation,citation_diff,citation_1yr,citation_2yr,citation_5yr,citation_10yr
0,0,1388,1.390561,11859.0,302,7688,3.0,2.0,333158.00,38.0,...,2.757852,1,[1976644430],[2011],1.0,0.0,0.0,0.0,0.0,0.0
1,1,49268,1.677395,11859.0,302,7688,170.0,30.0,351701.50,194.0,...,10.353863,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,209002,1.068871,11859.0,302,7688,1.0,0.0,1385029.00,0.0,...,1.068871,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,252754,1.083049,11859.0,302,7688,114.5,25.0,312797.75,194.0,...,9.060297,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,526827,0.722842,11859.0,302,7688,1.0,1.0,642772.50,8.0,...,0.722842,8,"[1558160053, 1981550344, 1998786313, 2952127479]","[2002, 2008, 2009, 2012]",4.0,4.0,1.0,1.0,1.0,4.0


In [21]:
# Set values for train dataset columns 
# train = pd.DataFrame(data=train.iloc[:,2:14].values, columns=["diversity","venue_rank","venue_MPI","venue_TPI","productivity","H_index","author_rank","author_MPI","author_TPI","versatility","n_citation"])
# Set ytrain and Xtrain
y_train = train.iloc[:,20]
X_train = train.iloc[:,2:12]
X_train.head()

,diversity,venue_rank,venue_MPI,venue_TPI,productivity,H_index,author_rank,author_MPI,author_TPI,versatility
0,1.390561,11859.0,302,7688,3.0,2.0,333158.00,38.0,62.0,2.757852
1,1.677395,11859.0,302,7688,170.0,30.0,351701.50,194.0,3250.0,10.353863
2,1.068871,11859.0,302,7688,1.0,0.0,1385029.00,0.0,0.0,1.068871
3,1.083049,11859.0,302,7688,114.5,25.0,312797.75,194.0,2376.5,9.060297
4,0.722842,11859.0,302,7688,1.0,1.0,642772.50,8.0,8.0,0.722842


In [22]:
y_train.head()

0    0.0
1    0.0
2    0.0
3    0.0
4    4.0
Name: citation_10yr, dtype: float64

In [23]:
# Set values for test dataset columns
# test = pd.DataFrame(data=test.iloc[:,2:14].values, columns=["diversity","venue_rank","venue_MPI","venue_TPI","productivity","H_index","author_rank","author_MPI","author_TPI","versatility","n_citation"])
# Set ytest and Xtest
y_test = test.iloc[:,20]
X_test = test.iloc[:,2:12]
X_test.head()

,diversity,venue_rank,venue_MPI,venue_TPI,productivity,H_index,author_rank,author_MPI,author_TPI,versatility
452,3.561625,11859.0,302,7688,15.500000,7.000000,482017.750000,63.500000,215.500000,10.951452
858,1.669966,11859.0,302,7688,25.000000,8.000000,566961.500000,68.000000,279.500000,11.066745
863,1.992607,11859.0,302,7688,19.000000,6.000000,773036.500000,16.000000,100.000000,7.633989
915,2.749466,11859.0,302,7688,7.333333,3.333333,841711.166667,14.333333,57.333333,6.582855
918,3.376685,11859.0,302,7688,18.333333,5.333333,815947.833333,25.333333,120.666667,10.587081


In [24]:
y_test.head()

452    3.0
858    1.0
863    5.0
915    3.0
918    1.0
Name: citation_10yr, dtype: float64

### Create model by dividing the data into train and test (or feature and label)

In [ ]:
# Creates the decision tree classifier object
clf = DecisionTreeClassifier(criterion='gini', splitter='best', max_depth = 25, min_samples_split=4, min_samples_leaf=2, random_state = 0)

# Train decision tree classifier
clf = clf.fit(X_train,y_train)

# Predicts response for test data
y_pred = clf.predict(X_test)

In [ ]:
# Model accuracy
print("Accuracy: ", metrics.accuracy_score(y_test,y_pred))